In [10]:
import os
import random

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Input
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, mean_absolute_error


In [11]:
DATADIR = "./Dataset"
TRAIN_TEST_CUTOFF = '2016-04-21'
TRAIN_VALID_RATIO = 0.75

In [12]:
# https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
# to implement F1 score for validation in a batch
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1macro(y_true, y_pred):
    f_pos = f1_m(y_true, y_pred)
    # negative version of the data and prediction
    f_neg = f1_m(1-y_true, 1-K.clip(y_pred,0,1))
    return (f_pos + f_neg)/2

def cnnpred_2d(seq_len=60, n_features=82, n_filters=(8,8,8), droprate=0.1):
    "2D-CNNpred model according to the paper"
    model = Sequential([
        Input(shape=(seq_len, n_features, 1)),
        Conv2D(n_filters[0], kernel_size=(1, n_features), activation="relu"),
        Conv2D(n_filters[1], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Conv2D(n_filters[2], kernel_size=(3,1), activation="relu"),
        MaxPool2D(pool_size=(2,1)),
        Flatten(),
        Dropout(droprate),
        Dense(1, activation="sigmoid")
    ])
    return model


In [13]:
def datagen(data, seq_len, batch_size, targetcol, kind):
    "As a generator to produce samples for Keras model"
    batch = []
    while True:
        # Pick one dataframe from the pool
        key = random.choice(list(data.keys()))
        df = data[key]
        input_cols = [c for c in df.columns if c != targetcol]
        index = df.index[df.index < TRAIN_TEST_CUTOFF]
        split = int(len(index) * TRAIN_VALID_RATIO)
        assert split > seq_len, "Training data too small for sequence length {}".format(seq_len)
        if kind == 'train':
            index = index[:split]   # range for the training set
        elif kind == 'valid':
            index = index[split:]   # range for the validation set
        else:
            raise NotImplementedError
        # Pick one position, then clip a sequence length
        while True:
            t = random.choice(index)     # pick one time step
            n = (df.index == t).argmax() # find its position in the dataframe
            if n-seq_len+1 < 0:
                continue # this sample is not enough for one sequence length
            frame = df.iloc[n-seq_len+1:n+1]
            batch.append([frame[input_cols].values, df.loc[t, targetcol]])
            break
        # if we get enough for a batch, dispatch
        if len(batch) == batch_size:
            X, y = zip(*batch)
            X, y = np.expand_dims(np.array(X), 3), np.array(y)
            yield X, y
            batch = []

def testgen(data, seq_len, targetcol):
    "Return array of all test samples"
    batch = []
    for key, df in data.items():
        input_cols = [c for c in df.columns if c != targetcol]
        # find the start of test sample
        t = df.index[df.index >= TRAIN_TEST_CUTOFF][0]
        n = (df.index == t).argmax()
        # extract sample using a sliding window
        for i in range(n+1, len(df)+1):
            frame = df.iloc[i-seq_len:i]
            batch.append([frame[input_cols].values, frame[targetcol][-1]])
    X, y = zip(*batch)
    return np.expand_dims(np.array(X),3), np.array(y)

In [14]:
# Read data into pandas DataFrames
data = {}
for filename in os.listdir(DATADIR):
    if not filename.lower().endswith(".csv"):
        continue # read only the CSV files
    filepath = os.path.join(DATADIR, filename)
    X = pd.read_csv(filepath, index_col="Date", parse_dates=True)
    # basic preprocessing: get the name, the classification
    # Save the target variable as a column in dataframe for easier dropna()
    name = X["Name"][0]
    del X["Name"]
    cols = X.columns
    X["Target"] = (X["Close"].pct_change().shift(-1) > 0).astype(int)
    X.dropna(inplace=True)
    # Fit the standard scaler using the training dataset
    index = X.index[X.index < TRAIN_TEST_CUTOFF]
    index = index[:int(len(index) * TRAIN_VALID_RATIO)]
    scaler = StandardScaler().fit(X.loc[index, cols])
    # Save scale transformed dataframe
    X[cols] = scaler.transform(X[cols])
    data[name] = X

seq_len = 60
batch_size = 128
n_epochs = 20
n_features = 82

In [15]:
# Produce CNNpred as a binary classification problem
model = cnnpred_2d(seq_len, n_features)
model.compile(optimizer="adam", loss="mae", metrics=["acc", f1macro])
model.summary()  # print model structure to console

# Set up callbacks and fit the model
# We use custom validation score f1macro() and hence monitor for "val_f1macro"
checkpoint_path = "./cp2d-{epoch}-{val_f1macro:.2f}.h5"
callbacks = [
    ModelCheckpoint(checkpoint_path,
                    monitor='val_f1macro', mode="max",
                    verbose=0, save_best_only=True, save_weights_only=False, save_freq="epoch")
]
model.fit(datagen(data, seq_len, batch_size, "Target", "train"),
          validation_data=datagen(data, seq_len, batch_size, "Target", "valid"),
          epochs=n_epochs, steps_per_epoch=100, validation_steps=10, verbose=1, callbacks=callbacks)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 60, 1, 8)          664       
                                                                 
 conv2d_4 (Conv2D)           (None, 58, 1, 8)          200       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 29, 1, 8)          0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 27, 1, 8)          200       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 13, 1, 8)          0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 104)              

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 64s 644ms/step - loss: 0.4311 - acc: 0.5685 - f1macro: 0.3621 - val_loss: 0.4946 - val_acc: 0.5055 - val_f1macro: 0.3353
Epoch 3/20
  4/100 [>.............................] - ETA: 2s - loss: 0.4353 - acc: 0.5684 - f1macro: 0.3665

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 68s 684ms/step - loss: 0.4306 - acc: 0.5752 - f1macro: 0.4111 - val_loss: 0.4856 - val_acc: 0.5180 - val_f1macro: 0.3405
Epoch 4/20
  5/100 [>.............................] - ETA: 3s - loss: 0.4257 - acc: 0.5828 - f1macro: 0.4630

...

100/100 [==============================] - 64s 646ms/step - loss: 0.2911 - acc: 0.7302 - f1macro: 0.7091 - val_loss: 0.4860 - val_acc: 0.5195 - val_f1macro: 0.4608
Epoch 8/20
100/100 [==============================] - 53s 531ms/step - loss: 0.2765 - acc: 0.7439 - f1macro: 0.7245 - val_loss: 0.4873 - val_acc: 0.5117 - val_f1macro: 0.4588
Epoch 9/20
100/100 [==============================] - 63s 641ms/step - loss: 0.2597 - acc: 0.7550 - f1macro: 0.7371 - val_loss: 0.4934 - val_acc: 0.5078 - val_f1macro: 0.4547
Epoch 10/20
100/100 [==============================] - 55s 557ms/step - loss: 0.2507 - acc: 0.7656 - f1macro: 0.7499 - val_loss: 0.5124 - val_acc: 0.4812 - val_f1macro: 0.4435
Epoch 11/20
100/100 [==============================] - ETA: 0s - loss: 0.2398 - acc: 0.7758 - f1macro: 0.7612

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 53s 540ms/step - loss: 0.2398 - acc: 0.7758 - f1macro: 0.7612 - val_loss: 0.4658 - val_acc: 0.5383 - val_f1macro: 0.4989
Epoch 12/20
100/100 [==============================] - 70s 708ms/step - loss: 0.2412 - acc: 0.7727 - f1macro: 0.7590 - val_loss: 0.5205 - val_acc: 0.4680 - val_f1macro: 0.4416
Epoch 13/20
100/100 [==============================] - ETA: 0s - loss: 0.2277 - acc: 0.7863 - f1macro: 0.7727

/usr/local/lib/python3.8/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


100/100 [==============================] - 55s 552ms/step - loss: 0.2277 - acc: 0.7863 - f1macro: 0.7727 - val_loss: 0.4884 - val_acc: 0.5203 - val_f1macro: 0.5026
Epoch 14/20
100/100 [==============================] - 64s 647ms/step - loss: 0.2243 - acc: 0.7905 - f1macro: 0.7796 - val_loss: 0.4840 - val_acc: 0.5148 - val_f1macro: 0.4903
Epoch 15/20
100/100 [==============================] - 62s 621ms/step - loss: 0.2136 - acc: 0.8005 - f1macro: 0.7888 - val_loss: 0.4774 - val_acc: 0.5234 - val_f1macro: 0.5006
Epoch 16/20
100/100 [==============================] - 52s 527ms/step - loss: 0.2219 - acc: 0.7916 - f1macro: 0.7803 - val_loss: 0.4892 - val_acc: 0.5156 - val_f1macro: 0.4707
Epoch 17/20
100/100 [==============================] - 53s 537ms/step - loss: 0.2086 - acc: 0.8045 - f1macro: 0.7950 - val_loss: 0.5004 - val_acc: 0.4930 - val_f1macro: 0.4673
Epoch 18/20
100/100 [==============================] - 65s 660ms/step - loss: 0.2027 - acc: 0.8078 - f1macro: 0.7961 - val_loss: 0.4

In [16]:
# Prepare test data
test_data, test_target = testgen(data, seq_len, "Target")

In [17]:
# Test the model
test_out = model.predict(test_data)
test_pred = (test_out > 0.5).astype(int)
print("accuracy:", accuracy_score(test_pred, test_target))
print("MAE:", mean_absolute_error(test_pred, test_target))
print("F1:", f1_score(test_pred, test_target))

33/33 [==============================] - 0s 4ms/step
accuracy: 0.5365853658536586
MAE: 0.4634146341463415
F1: 0.6575342465753425
